# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generate-pre-processed-FASTA-files" data-toc-modified-id="Generate-pre-processed-FASTA-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate pre-processed FASTA files</a></div><div class="lev2 toc-item"><a href="#Setup" data-toc-modified-id="Setup-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Prepare-FASTA" data-toc-modified-id="Prepare-FASTA-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prepare FASTA</a></div><div class="lev1 toc-item"><a href="#Simulate-MS-data" data-toc-modified-id="Simulate-MS-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simulate MS-data</a></div><div class="lev2 toc-item"><a href="#Prepare-quantity-matrix" data-toc-modified-id="Prepare-quantity-matrix-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare quantity matrix</a></div>

# Generate pre-processed FASTA files

Using the custom-made `seqtk generate_spikein.py` script.

Dependencies:

* OpenMS binaries in PATH (version 2.2.0)

## Setup

In [1]:
threads=6
run=batch14_fold2
mkdir ${run}

In [2]:

echo -e "name\tbiorepgroup\ttechrepgroup\tcondition\tbatch" > ${run}/design.tsv
echo -e "a1\t1\t1\ta\t1" >> ${run}/design.tsv
echo -e "a2\t2\t1\ta\t1" >> ${run}/design.tsv
echo -e "a3\t3\t1\ta\t1" >> ${run}/design.tsv
echo -e "a4\t4\t1\ta\t2" >> ${run}/design.tsv
echo -e "a5\t5\t1\ta\t2" >> ${run}/design.tsv
echo -e "a6\t6\t1\ta\t2" >> ${run}/design.tsv
echo -e "b1\t1\t1\tb\t1" >> ${run}/design.tsv
echo -e "b2\t2\t1\tb\t1" >> ${run}/design.tsv
echo -e "b3\t3\t1\tb\t1" >> ${run}/design.tsv
echo -e "b4\t4\t1\tb\t2" >> ${run}/design.tsv
echo -e "b5\t5\t1\tb\t2" >> ${run}/design.tsv
echo -e "b6\t6\t1\tb\t2" >> ${run}/design.tsv



In [3]:
cat ${run}/design.tsv

name	biorepgroup	techrepgroup	condition	batch
a1	1	1	a	1
a2	2	1	a	1
a3	3	1	a	1
a4	4	1	a	2
a5	5	1	a	2
a6	6	1	a	2
b1	1	1	b	1
b2	2	1	b	1
b3	3	1	b	1
b4	4	1	b	2
b5	5	1	b	2
b6	6	1	b	2


In [4]:
sample_names=$(cut -f1 ${run}/design.tsv \
    | tr "\n" " " | cut -f2- -d" ")
file_names=$(cut -f1 ${run}/design.tsv \
    | tr "\n" " " | cut -f2- -d" " \
    | sed "s/ /.fa,/g" | sed "s/,$//")
batch=$(cut -f2 ${run}/design.tsv | tr "\n" " " \
    | cut -f2- -d" ")

echo ${sample_names}
echo ${file_names}
echo ${batch}

a1 a2 a3 a4 a5 a6 b1 b2 b3 b4 b5 b6
a1.fa,a2.fa,a3.fa,a4.fa,a5.fa,a6.fa,b1.fa,b2.fa,b3.fa,b4.fa,b5.fa,b6.fa
1 2 3 4 5 6 1 2 3 4 5 6


## Prepare FASTA

In [5]:
util_scripts/lfqtk/lfqtk.py generate_spikein_set \
    --background_fa data/uniprot_ecoli.pure.fasta \
    --spikein_fa data/uniprot_potato.pure.fasta \
    --offset_mean 500000 \
    --offset_std 0 \
    --base_int 1000000 \
    --noise_std 200000 \
    --back_count 50 \
    --spike_count 10 \
    --spike_folds "1,1,1,1,1,1,2,2,2,2,2,2" \
    --offset_folds "0,0,0,1,1,1,0,0,0,1,1,1" \
    --out_base ${run} \
    --sample_names ${file_names} \
    --verbose

4436 entries loaded from data/uniprot_ecoli.pure.fasta as background
403 entries loaded from data/uniprot_potato.pure.fasta as spike-in
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/a1.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/a2.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/a3.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/a4.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/a5.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/a6.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/b1.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/b2.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch14_fold2/b3.fa
50 entries picked as background, 10 as spike-in
60 ent

# Simulate MS-data

Generate the OpenMS tool `MSSimulator` for prepared FASTA files with assigned intensities.

In [6]:
time parallel -j ${threads} \
"echo \"Processing {}\"
MSSimulator \
    -in ${run}/{}.fa \
    -out_fm ${run}/{}.ground.featureXML" \
    ::: ${sample_names}


Processing a4
Loading sequence data from batch14_fold2/a4.fa ...
done (60 protein(s) loaded)
Starting simulation
2017/11/10, 15:44:17: Digest Simulation ... started
2017/11/10, 15:44:17: RT Simulation ... started
2017/11/10, 15:44:19: Predicting RT ... done
2017/11/10, 15:44:19: RT prediction gave 'invalid' results for 1698 peptide(s), making them unobservable.
2017/11/10, 15:44:19:   (List is too big to show)
2017/11/10, 15:44:19: Creating experiment with #501 scans ... done
2017/11/10, 15:44:19: Detectability Simulation ... started
2017/11/10, 15:44:19: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/10, 15:44:19: Simulating 1959 features
Progress of 'Ionization':
-- done [took 14.16 s (CPU), 14.25 s (Wall)] -- 
2017/11/10, 15:44:34: #Peptides not ionized: 0
2017/11/10, 15:44:34: #Peptides outside mz range: 731
2017/11/10, 15:44:34: Raw MS1 Simulation ... started
2017/11/10, 15:44:34:   Simulating signal for 5475 features ...
Progress of 'Ra

2017/11/10, 15:44:43: Contaminants out-of-RT-range: 204 / 486
2017/11/10, 15:44:43: Contaminants out-of-MZ-range: 111 / 486
2017/11/10, 15:44:45: Compressed data to grid ... 10047999 --> 9030145 (89%)
2017/11/10, 15:44:45: Adding white noise to spectra ...
2017/11/10, 15:44:45: Adding detector noise to spectra ...
2017/11/10, 15:44:45: Detector noise was disabled.
2017/11/10, 15:44:45: Tandem MS Simulation ... disabled
2017/11/10, 15:44:45: Final number of simulated features: 5474
2017/11/10, 15:44:45: Simulation took 28.602088 seconds
2017/11/10, 15:44:45: Storing simulated features in: batch14_fold2/a5.ground.featureXML
2017/11/10, 15:44:46: MSSimulator took 29.19 s (wall), 28.91 s (CPU), 0.00 s (system), 28.91 s (user).
Processing a6
Loading sequence data from batch14_fold2/a6.fa ...
done (60 protein(s) loaded)
Starting simulation
2017/11/10, 15:44:17: Digest Simulation ... started
2017/11/10, 15:44:17: RT Simulation ... started
2017/11/10, 15:44:19: Predicting RT ... done
2017/11/1

2017/11/10, 15:44:48:   (List is too big to show)
2017/11/10, 15:44:49: Creating experiment with #501 scans ... done
2017/11/10, 15:44:49: Detectability Simulation ... started
2017/11/10, 15:44:49: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/10, 15:44:49: Simulating 1960 features
Progress of 'Ionization':
-- done [took 16.33 s (CPU), 16.37 s (Wall)] -- 
2017/11/10, 15:45:05: #Peptides not ionized: 0
2017/11/10, 15:45:05: #Peptides outside mz range: 739
2017/11/10, 15:45:05: Raw MS1 Simulation ... started
2017/11/10, 15:45:05:   Simulating signal for 5475 features ...
Progress of 'RawMSSignal':
-- done [took 6.72 s (CPU), 6.78 s (Wall)] -- 
2017/11/10, 15:45:13: Contaminants out-of-RT-range: 204 / 486
2017/11/10, 15:45:13: Contaminants out-of-MZ-range: 111 / 486
2017/11/10, 15:45:14: Compressed data to grid ... 10063523 --> 9036600 (89%)
2017/11/10, 15:45:14: Adding white noise to spectra ...
2017/11/10, 15:45:14: Adding detector noise to s

In [7]:
for xml in ${run}/*.featureXML; do 
    ProteinQuantifier \
        -in ${xml} \
        -peptide_out ${xml%.*}.csv
done


Processing summary - number of...
...features: 5475 used for quantification, 5475 total (0 no annotation, 0 ambiguous annotation)
...peptides: 1954 quantified, 1954 identified (considering best hits only)
ProteinQuantifier took 1.26 s (wall), 1.23 s (CPU), 0.00 s (system), 1.23 s (user).

Processing summary - number of...
...features: 5474 used for quantification, 5474 total (0 no annotation, 0 ambiguous annotation)
...peptides: 1954 quantified, 1954 identified (considering best hits only)
ProteinQuantifier took 1.25 s (wall), 1.23 s (CPU), 0.00 s (system), 1.23 s (user).

Processing summary - number of...
...features: 5475 used for quantification, 5475 total (0 no annotation, 0 ambiguous annotation)
...peptides: 1954 quantified, 1954 identified (considering best hits only)
ProteinQuantifier took 1.26 s (wall), 1.25 s (CPU), 0.00 s (system), 1.25 s (user).

Processing summary - number of...
...features: 5475 used for quantification, 5475 total (0 no annotation, 0 ambiguous annotation)

We receive identical peptide setup due to same protein being picked and cleaved (even if intensities varies).

## Prepare quantity matrix

In [8]:
python3 util_scripts/combine_quant_pd.py \
    --dfs ${run}/*.csv \
    --out_fp ${run}/full_quant.tsv

In [10]:
wc -l ${run}/full_quant.tsv
head ${run}/full_quant.tsv

1946 batch14_fold2/full_quant.tsv
peptide	protein	a1	a2	a3	a4	a5	a6	b1	b2	b3	b4	b5	b6
CLK	sp|P37127|AEGA_ECOLI	27859100.0	47199200.0	48350400.0	87030800.0	64006700.0	66078900.0	36854100.0	21648000.0	29196300.0	72337000.0	73013000.0	74365200.0
FCR	sp|P0A7T7|RS18_ECOLI	140711604.0	102465796.0	120203200.0	175086700.0	200098700.0	179798000.0	119185600.0	210603108.0	166641100.0	135449704.0	169127700.0	168260400.0
LCR	sp|Q9JMR4|YUBK_ECOLI	49510300.0	59627600.0	34226700.0	61134400.0	79216496.0	65654900.0	41219500.0	50987400.0	55255100.0	80637000.0	77535504.0	96989904.0
FCQR	sp|P39357|YJHF_ECOLI	144461200.0	112001696.0	140922100.0	206195008.0	225476008.0	166140600.0	235997400.0	85933000.0	89382100.0	226930208.0	133279800.0	205833292.0
FLFK	sp|P34094|PHYB_SOLTU	99838500.0	109387700.0	138451200.0	202539600.0	163332600.0	157562700.0	227359600.0	250199192.0	231104092.0	408702200.0	366921216.0	383277616.0
FYLS	sp|A5A617|YDGU_ECOLI	81157104.0	71129400.0	51876400.0	109368000.0	99474200.0	98939400.0	9